In [ ]:
class Select(object):
    _sql = "select {field} from {table} where {filter} {group}"

    def __init__(self, fld, flt, grp):
        self._field = fld
        self._filter = flt
        self._group = grp

    def __call__(self, fn):
        def wrapped(*args, **kwargs):
            fieldStr = ','.join(self._field)
            filterStr = ','.join(self._filter)
            groupStr = ','.join(self._group)
            groupStr = "" if len(groupStr) <= 0 else "group by " + groupStr
            return self._sql.format(field=fieldStr,
                                    table=fn(*args, **kwargs),
                                    filter=filterStr,
                                    group=groupStr)

        return wrapped


class SubQuery(object):
    _sql = "select * from ({subquery}) t"

    def __init__(self):
        pass

    def __call__(self, fn):
        def wrapped(*args, **kwargs):
            return self._sql.format(subquery=fn(*args, **kwargs))
        return wrapped

In [ ]:
@SubQuery()
@Select(fld=['*'], flt=["1=1"], grp=[])
def create(table):
    return table

s=create("t1")
print(s)

In [ ]:
class Join(object):
    _sql = """
    select {field} from ({t1}) t1
    join ({t2}) t2 on({on})
    """

    def __init__(self,field,t1,t2,on):
        self._t1=t1
        self._t2=t2
        self._on=on

    def __call__(self, fn):
        def wrapped(*args, **kwargs):
            return self._sql.format(t1=self._t1,t2=self._t2,on=self._on)
        return wrapped